# ChatGLM Prompt
本Notebook运行于本机（Mac）

本节中，将使用提示工程的方式基于ChatGLM生成标题。具体来说分为以下三个方法：
1. 输入正文，直接生成标题
2. 基于Langchain，多次生成标题后再汇总生成标题
3. 基于Langchain，使用训练集构建向量数据库，根据新闻正文相似性查找对应的训练数据作为“例子”，再生成标题


### ChatGLM直接生成标题
ChatGLM运行在服务器中，采用官方仓库中的API进行调用，默认使用8000端口，以Post方式传入对话数据。
> https://github.com/THUDM/ChatGLM-6B

同时使用Langchain进行二次封装，方便之后的调用与功能实现.
> https://python.langchain.com/en/latest/index.html

In [2]:
# 借助Langchain框架实现ChatGLM的API调用，方便之后使用Prompt
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from typing import Dict, List, Optional, Tuple, Union
import requests
import json
class ChatGLM(LLM):
    max_token: int = 2048
    temperature: float = 0.1
    top_p = 0.9
    history = []
    def __init__(self):
        super().__init__()
    @property
    def _llm_type(self) -> str:
        return "ChatGLM"
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        headers = {'Content-Type': 'application/json'}
        data=json.dumps({
          'prompt':prompt,
          'temperature':self.temperature,
          'history':self.history,
          'max_length':self.max_token
        })
        # print("ChatGLM prompt:",prompt)
        # 调用api
        success = False
        failure_count = 0
        while not success and failure_count < 4:
            try:
                response = requests.post("http://192.168.192.160:8000",headers=headers,data=data,timeout=120)
                if response.status_code==200:
                    success = True
                    resp = response.json()
                    if stop is not None:
                        response = enforce_stop_tokens(response, stop)
                    self.history = self.history+[[None, resp['response']]]
                    return resp['response']
                else:
                    failure_count += 1
            except:
                failure_count += 1
                continue
        return "ChatGLM调用失败"
            

In [2]:
# 测试一下效果
chatglm = ChatGLM()
print(chatglm("请介绍一下你自己"))


我是一个名为 ChatGLM-6B 的人工智能助手，是基于清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。我是一段程序，并没有自我意识，也不能像人类一样感知世界。


In [3]:
# 读取测试集数据
import pandas as pd


df_raw_test = pd.read_csv("data/test_data.csv")
        
title,context = df_raw_test.iloc[0,0],df_raw_test.iloc[0,1]
title,context

('刘烨儿子取名诺一：生个女儿就叫“千金”',
 '\u3000\u300032岁的刘烨当爸了，给儿子取名“诺一”。昨天，从法国风尘仆仆赶回、直奔《爱出色》官网启动式现场的刘烨，从头到尾都在兴奋：“取名诺一，是因为‘一诺千金’。我跟媳妇早就说好了，生个儿子就叫‘诺一’，生个女儿就叫‘千金’！”\n\u3000\u30009月底，刘烨刚拍完《建党伟业》的戏份，就飞赴法国陪妻子安娜待产，10月10日，他们的混血儿子“诺一”降临人世，“自己有了孩子，才知道当父母有多累。我这个儿子还特别能哭闹，害得我晚上经常睡不好觉。”当上爹的刘烨，更念“父母恩”：“以前我是一周给父母打个电话，有了孩子后，每天都要跟他们联系。我还不断问我妈很幼稚的问题，比如‘我小时候是不是也像小动物似的趴在你们身上一动不动，没父母就活不了？’”本报驻北京记者 梅春艳\n\u3000\u3000《建党伟业》小凤仙\n\u3000\u3000最终花落Angelababy\n\u3000\u3000献礼片《建党伟业》明星众多，不过由谁来扮演同“护国将军”蔡锷有过一段千古之情的小凤仙，一直引人关注。之前被曝已定下由范冰冰出演这一角色。不过昨日，中影对外发布消息称，范冰冰由于档期不合出演不成，这一角色最终花落香港嫩模出身的Angelababy。近日，剧组转站河北迁安，出演蔡锷的刘德华以及Angelababy也相继入组。\n\u3000\u3000据了解，剧组在迁安所拍摄的主要是小凤仙送别蔡锷的戏份，为了找到符合当年情景的老火车以及火车站，剧组可谓费尽心思，一直找到河北迁安。据影片摄影指导赵晓介绍，迁安的火车站机身和车厢保存比较完整，也比较古老，再加上美术、置景与特技，能够最大程度上还原当年小凤仙送别蔡锷的场景。\n\u3000\u3000据剧组介绍，Angelababy最初一直进不了状态，还好刘德华经验很足，主动与Angelababy说戏，悉心引导，好在Angelababy完全放开后，表现不错。记者 冯泽\n\n')

具体的Prompt设置方面，这里也先进行了一些尝试，发现ChatGLM在某些较短的新闻内容生成标题时效果还不错，但是对一些较长的新闻内容往往会生成过于长的新闻标题。因此，增加“写一个20字以内”的Prompt，但是实际生成时，某些标题长度依然超过了20字。

In [4]:
# 利用Langchain的Prompt Templates与Chains构造Prompt并生成标题
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["context"],
    template="请根据新闻内容帮我写一个20字以内的新闻短标题：{context}?",
)
direct_generate_chain = LLMChain(llm=chatglm, prompt=prompt)
direct_generate_chain.run(context)



'刘烨给儿子取名“诺一”，范冰冰因档期不合退出《建党伟业》扮演小凤仙的角色，最终由Angelababy饰演。'

In [3]:
# 利用Langchain的Prompt Templates与Chains构造Prompt并生成标题
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 获取chatglm的chain
def get_chatglm_chain(prompt):
    chatglm = ChatGLM()
    chain = LLMChain(llm=chatglm, prompt=prompt)
    return chain


def format_generate_result(result):
    # 通过测试集数据发现可能存在以下情况：“新闻标题是：\n”、“新闻短标题是：”，或者直接列出多个标题，或者生成一长段话
    # 删除句号及句号之后的内容,处理生成一长段的情况
    result = result.split("。")[0]
    # 处理“新闻标题是：\n”、“新闻短标题是：”的情况
    if "：\n" in result and "标题" in result:
        result = result.split("：\n")[1]
    elif "：" in result and "标题" in result:
        result = result.split("：")[1]
    # 处理多个标题的情况
    result = result.split("\n")[0]    
    return result


from tqdm import tqdm
# 读取数据与保存数据
def run(name,generate_method,save_path = "",read_path = 'data/test_data.csv',saving_step=20,vector_db = None,test_mode=False,append=False):
    df = pd.read_csv(read_path)
    if not append or name not in df.columns:
        df[name] = pd.Series()
    if test_mode:
        max_iter = 5
        saving_step = 1
    else: 
        max_iter = len(df)
    for i in tqdm(range(max_iter)):
        if df[name].isna()[i] or df[name].iloc[i] == "ChatGLM调用失败" :
            title,context = df.iloc[i,0],df.iloc[i,1]
            if vector_db is not None:
                result = generate_method(context,vector_db)
            else:
                result = generate_method(context)
            result = format_generate_result(result)
            if test_mode :
                print(f'{title}-----{result}')
            else:
                df[name].iloc[i] = result
                if save_path != "":
                    if i%saving_step==0 or i==max_iter-1:
                        df.to_csv(save_path,index=False)
    return df

def run_test(name,generate_method,vector_db=None):
    run(name,generate_method,"",vector_db=vector_db,test_mode=True)
    


In [4]:
# 直接生成标题

def generate_prompt_direcet_title(context):
    prompt = PromptTemplate(
    input_variables=["context"],
    template="请根据新闻内容帮我写一个20字以内的新闻短标题，标题要能简洁明了概括新闻、吸引读者、具有文采：{context}?",
    )
    chain = get_chatglm_chain(prompt)
    result = chain.run(context)
    return result

    

In [14]:
# 先测试一下效果
run_test(name="prompt_direct_title",generate_method = generate_prompt_direcet_title)


 20%|██        | 1/5 [00:04<00:18,  4.61s/it]

刘烨儿子取名诺一：生个女儿就叫“千金”-----刘烨儿子命名“诺一”，范冰冰落选《建党伟业》小凤仙角色；Angelababy入组《建党伟业》，剧组在河北迁安拍摄还原场景


 40%|████      | 2/5 [00:07<00:11,  3.82s/it]

乡村乐组合Sugarland遭前队友索赔1400万美元-----Sugarland组合与前队友克里斯汀-霍尔之间的财务诉讼庭审成功和解


 60%|██████    | 3/5 [00:15<00:11,  5.57s/it]

彩民周刊10139期双色球：一区2路号已亟待回补-----1路号07、10概率较大，4枚1路号或2枚2路号；0路号33删除；25、15、22、27、28重点防；0路号25、17、32仅保留1-2枚；01、21、23、26、30有重号开出


 80%|████████  | 4/5 [00:18<00:04,  4.71s/it]

女子锤杀出轨丈夫割下其生殖器(图)-----女子锤杀丈夫并割下其生殖器，法院轻判凶手14年


100%|██████████| 5/5 [00:21<00:00,  4.23s/it]

北大考研记：打开双手世界就在你手中-----考生坚持考研，梦想实现在望


### Langchain多次生成标题

使用Langchain构建多个ChatGLM的Agent，其中一个Agent负责汇总其余Agent的标题生成结果，最终生成标题。

具体Prompt尝试过：
1. 请根据新闻内容帮我写一个20字以内的新闻短标题：{context}
2. 请根据新闻内容帮我写一个新闻短标题，一定要简短：{context}
3. 请根据新闻内容帮我写一个能吸引读者兴趣的新闻短标题：{context}
4. 你现在是一个资深的新闻编辑，请根据以下内容拟一个易读的新闻短标题：{context}
5. 请根据新闻内容帮我写一个能简洁明了概括具体内容的新闻短标题：\n{context}
6. 请根据新闻内容帮我写一个能引人入胜，吸引阅读的新闻短标题：\n{context}
7. 请根据以下新闻内容写一个文采斐然，具有内涵的新闻短标题：\n{context}
8. 请你作为资深的新闻编辑，参考以下几个新闻标题，生成一个更好的短标题给我（不要给我选择，只要生成一个更好的新闻短标题）：\n{context}
9. 以下是几个新闻标题，请你作为资深的新闻编辑，给出一个更好的标题：{context}

In [5]:
def generate_prompt_combined_title(context):
    prompt_a2 = PromptTemplate(
        input_variables=["context"],
        template="请根据新闻内容帮我写一个能简洁明了概括具体内容的新闻短标题：\n{context}",
    )
    prompt_a3 = PromptTemplate(
        input_variables=["context"],
        template="请根据新闻内容帮我写一个能引人入胜，吸引阅读的新闻短标题：\n{context}",
    )
    prompt_a4 = PromptTemplate(
        input_variables=["context"],
        template="请根据以下新闻内容写一个文采斐然，具有内涵的新闻短标题：\n{context}",
    )
    prompt_summarize = PromptTemplate(
        input_variables=["context"],
        template="请你作为资深的新闻编辑，参考以下几个新闻标题，选择一个最好的短标题给我：\n{context}",
    )
    prompt_too_long = PromptTemplate(
        input_variables=["context"],
        template="请将以下内容总结概括，生成一个20字以内的新闻标题：\n{context}",
    )
    prompt_list = [prompt_a2,prompt_a3,prompt_a4]
    agents = []
    results_of_agents = ''
    for prompt in prompt_list:
        chain = get_chatglm_chain(prompt)
        agents.append(chain)
    summarize_agent = get_chatglm_chain(prompt_summarize)
    short_agent = get_chatglm_chain(prompt_too_long)
    for agent in agents:
        result = agent.run(context)
        if len(result) > 35: # 还是会有生成标题过长的情况出现，所以这里手动增加了判断逻辑，如果生成的标题过长，就再总结一次（最开始是直接丢弃，但发现会出现所有标题都过长的情况）
            result = short_agent.run(result)
        results_of_agents += result + '\n'
    results_of_summarize = summarize_agent.run(results_of_agents)
    return results_of_summarize
        

In [26]:
run_test(name="prompt_combined_title",generate_method = generate_prompt_combined_title)


 20%|██        | 1/5 [00:46<03:06, 46.68s/it]

刘烨儿子取名诺一：生个女儿就叫“千金”-----刘烨儿子命名“诺一”,Angelababy成《建党伟业》小凤仙角色之一，迁安戏份能够还原角色


 40%|████      | 2/5 [01:19<01:55, 38.59s/it]

乡村乐组合Sugarland遭前队友索赔1400万美元-----Sugarland组合前成员起诉前队友，庭外和解，要求分享组合利润的三分之一


 60%|██████    | 3/5 [04:07<03:15, 97.55s/it]

彩民周刊10139期双色球：一区2路号已亟待回补-----


 80%|████████  | 4/5 [04:48<01:15, 75.39s/it]

女子锤杀出轨丈夫割下其生殖器(图)-----女子锤杀丈夫被判14年，从轻处罚


100%|██████████| 5/5 [05:06<00:00, 61.34s/it]

北大考研记：打开双手世界就在你手中-----考生北大情缘：紧握双手握世界，考研成功触摸梦想


### Langchain知识库+One shot


In [8]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import pandas as pd



In [10]:
def create_and_save_vector_db():
    df_val_result = pd.read_csv("data/train_data.csv")
    context,title = df_val_result['context'].tolist(),df_val_result['title'].tolist()
    text = ['示例新闻内容：'+context[i]+'\n\n示例新闻标题：'+title[i] for i in range(len(title))]
    embeddings = HuggingFaceEmbeddings(model_name="GanymedeNil/text2vec-large-chinese",model_kwargs={"device": "mps"}) #本机运行环境为MacOS，调用苹果的ML加速，device为mps，如果是Linux或者Windows，可以改为cuda
    db = FAISS.from_texts(text, embeddings)
    db.save_local("vector_db")
    
    
def load_vector_db():
    embeddings = HuggingFaceEmbeddings(model_name="GanymedeNil/text2vec-large-chinese",model_kwargs={"device": "mps"})
    db = FAISS.load_local("vector_db", embeddings)
    return db


def get_example(db,query):
    query_result = db.similarity_search(query, k=1)
    return query_result[0].page_content


def generate_prompt_example_title(context,db):
    prompt = PromptTemplate(
        input_variables=["context","example"],
        template="{example}\n\n请参照上面的示例，帮我写一个20字以内的新闻短标题，标题要能简洁明了概括新闻、吸引读者、具有文采。\n新闻内容：{context}",
    )
    
    example = get_example(db,context)
    chain = get_chatglm_chain(prompt)
    result = chain.run({'context':context,'example':example})
    return result
        

In [19]:
# 加载与保存数据库
# create_and_save_vector_db()
# 耗时15分34秒


In [11]:
# 测试一下向量数据库查询
query = "明星八卦"
db = load_vector_db()
print(get_example(db,query))

/Users/tianminghao/miniforge3/envs/title-generate/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name /Users/tianminghao/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese. Creating a new one with MEAN pooling.


示例新闻内容：　　文/泡菜先锋
　　【新浪游戏独家专稿，转载请注明出处】
　　近日，韩服《C9》(国服名：第九大陆)推出了第五大陆后半部内容视频，以吸引玩家们的眼球。
　　《第九大陆》是一款全3D动作MMORPG，由NHNGames公司从2006年开始策划及开发。游戏画面效果出色，且具有出色的人工智能怪物，与玩家进行对抗。《第九大陆》自2009年8月起在韩国运营，其公测首日就吸引了42万余人前来体验。在2009年韩国年度游戏评奖典礼上更是包揽了总统奖、优秀开发者、图形、角色、音效5项大奖。
　　《第九大陆》国服由腾讯游戏运营。



示例新闻标题：韩服《C9》第五大陆后半部内容视频


In [27]:
run_test(name="prompt_example_title",generate_method = generate_prompt_example_title,vector_db=db)
# 耗时51秒

 20%|██        | 1/5 [00:08<00:34,  8.54s/it]

刘烨儿子取名诺一：生个女儿就叫“千金”-----张柏芝产二胎 谢霆锋陪产剪脐带 媒体苦等消息


 40%|████      | 2/5 [00:13<00:18,  6.24s/it]

乡村乐组合Sugarland遭前队友索赔1400万美元-----美国乡村乐组合 Sugarland 与前队友达成和解，避免财务诉讼


 60%|██████    | 3/5 [00:18<00:12,  6.01s/it]

彩民周刊10139期双色球：一区2路号已亟待回补-----双色球第10139期奖号预测


 80%|████████  | 4/5 [00:22<00:05,  5.24s/it]

女子锤杀出轨丈夫割下其生殖器(图)-----男子锤杀妻子并割下其生殖器后自首，法院轻判凶手14年


100%|██████████| 5/5 [00:26<00:00,  5.25s/it]

北大考研记：打开双手世界就在你手中-----白羊座男友大男子主义严重，女友心灵孤单


## 生成数据

In [12]:
test_result_path = "result/test_result.csv"
val_result_path = "result/val_result.csv"

In [28]:
# 创建保存结果的csv
def create_result_csv(path,raw_csv_path):
    df_raw = pd.read_csv(raw_csv_path)
    df_result = pd.DataFrame(columns=['title','context'])
    df_result['title'] = df_raw['title']
    df_result['context'] = df_raw['context']
    df_result.to_csv(path,index=False)


create_result_csv("result/test_result.csv","data/test_data.csv")
create_result_csv("result/val_result.csv","data/val_data.csv")


### 测试集

In [29]:
run(name="chatglm_one_shot",generate_method = generate_prompt_example_title,vector_db=db,read_path=test_result_path,save_path=test_result_path)


100%|██████████| 200/200 [21:10<00:00,  6.35s/it]


,title,context,chatglm_one_shot
0,刘烨儿子取名诺一：生个女儿就叫“千金”,32岁的刘烨当爸了，给儿子取名“诺一”。昨天，从法国风尘仆仆赶回、直奔《爱出色》官网启动...,张柏芝产二胎 谢霆锋陪产剪脐带 媒体苦等消息
1,乡村乐组合Sugarland遭前队友索赔1400万美元,新浪娱乐讯 北京时间11月15日消息，据国外媒体报道，美国乡村乐双人组合Sugarlan...,美国乡村乐组合 Sugarland 与前队友达成和解，避免财务诉讼
2,彩民周刊10139期双色球：一区2路号已亟待回补,综述下期遗漏值\n 一、遗漏1期号码07、09、10、13、19、33。\n 上述号...,双色球第10139期奖号预测
3,女子锤杀出轨丈夫割下其生殖器(图),本报讯 昨日，漳州龙文区法院对“女子锤死丈夫并割下其生殖器”一案进行一审判决，被告人王某...,男子锤杀妻子并割下其生殖器后自首，法院轻判凶手14年
4,北大考研记：打开双手世界就在你手中,天空映射着未名湖的秀美，漫步在湖边小路，此情此景，刺激我敏感的思绪奔越进数年前，回想与北...,白羊座男友大男子主义严重，女友心灵孤单
...,...,...,...
195,基金公司海捞专户投资经理,证券时报记者 程俊琳\n 本报讯 随着多元化业务的拓展，基金公司的人才缺口越来越大。面...,1. 基金“一对多”业务在国内铺开，券商集合理财加入混战
196,汉堡主场输球难倒众媒体--胜负彩09041期媒体比对,北京时间05月17日，中国足球彩票中国足球彩票14场胜负游戏第09041期比赛结束，本期...,媒体推荐历史命中率统计比对，新浪体育提供参考
197,徐若�富二代男友：以结婚为前提交往 已见家长,徐若?和富二代贺锡敬恋情曝光，25日贺锡敬出席活动时，被逼问和徐若?的交往情况，他松口表...,徐若？和富二代贺锡敬公开恋情，两人已见过家长，但徐若？太忙成“空中飞人”
198,三个亿元奖皆被神秘兑走 仅广州9668万得主露真容,中国人一向信奉“人怕出名猪怕壮”，中了彩票巨奖后的兴奋可想而知，但如何躲过媒体耳目成功兑...,最淡定的500万大奖得主：神秘现身，仅一人领奖


In [30]:
run(name="chatglm_zero_shot", generate_method=generate_prompt_direcet_title, read_path=test_result_path, save_path=test_result_path)

100%|██████████| 200/200 [16:11<00:00,  4.86s/it] 


,title,context,chatglm_one_shot,chatglm_zero_shot
0,刘烨儿子取名诺一：生个女儿就叫“千金”,32岁的刘烨当爸了，给儿子取名“诺一”。昨天，从法国风尘仆仆赶回、直奔《爱出色》官网启动...,张柏芝产二胎 谢霆锋陪产剪脐带 媒体苦等消息,刘烨儿子命名“诺一”，范冰冰落选《建党伟业》小凤仙角色；Angelababy入组《建党伟业》...
1,乡村乐组合Sugarland遭前队友索赔1400万美元,新浪娱乐讯 北京时间11月15日消息，据国外媒体报道，美国乡村乐双人组合Sugarlan...,美国乡村乐组合 Sugarland 与前队友达成和解，避免财务诉讼,Sugarland组合与前队友克里斯汀-霍尔之间的财务诉讼庭审成功和解
2,彩民周刊10139期双色球：一区2路号已亟待回补,综述下期遗漏值\n 一、遗漏1期号码07、09、10、13、19、33。\n 上述号...,双色球第10139期奖号预测,下期遗漏值综述：1路号07、10概率较大，4枚1路号或2枚2路号；0路号33删除；25、15...
3,女子锤杀出轨丈夫割下其生殖器(图),本报讯 昨日，漳州龙文区法院对“女子锤死丈夫并割下其生殖器”一案进行一审判决，被告人王某...,男子锤杀妻子并割下其生殖器后自首，法院轻判凶手14年,女子锤杀丈夫并割下其生殖器，法院轻判凶手14年
4,北大考研记：打开双手世界就在你手中,天空映射着未名湖的秀美，漫步在湖边小路，此情此景，刺激我敏感的思绪奔越进数年前，回想与北...,白羊座男友大男子主义严重，女友心灵孤单,考生坚持考研，梦想实现在望
...,...,...,...,...
195,基金公司海捞专户投资经理,证券时报记者 程俊琳\n 本报讯 随着多元化业务的拓展，基金公司的人才缺口越来越大。面...,1. 基金“一对多”业务在国内铺开，券商集合理财加入混战,基金公司人才缺口大，券商、保险等金融机构被挖角
196,汉堡主场输球难倒众媒体--胜负彩09041期媒体比对,北京时间05月17日，中国足球彩票中国足球彩票14场胜负游戏第09041期比赛结束，本期...,媒体推荐历史命中率统计比对，新浪体育提供参考,新浪体育比较多家媒体投注推介，发现其中一些媒体的历史命中率较低
197,徐若�富二代男友：以结婚为前提交往 已见家长,徐若?和富二代贺锡敬恋情曝光，25日贺锡敬出席活动时，被逼问和徐若?的交往情况，他松口表...,徐若？和富二代贺锡敬公开恋情，两人已见过家长，但徐若？太忙成“空中飞人”,徐若？和富二代贺锡敬公开恋情，已见过家长，相处轻松
198,三个亿元奖皆被神秘兑走 仅广州9668万得主露真容,中国人一向信奉“人怕出名猪怕壮”，中了彩票巨奖后的兴奋可想而知，但如何躲过媒体耳目成功兑...,最淡定的500万大奖得主：神秘现身，仅一人领奖,中国彩市诞生三大亿元巨奖，大奖得主如何躲过媒体目光成功兑奖及不公开真容？


In [31]:
run(name="chatglm_least_to_most", generate_method=generate_prompt_combined_title, read_path=test_result_path, save_path=test_result_path)

  9%|▉         | 18/200 [15:36<2:24:58, 47.79s/it]

### 验证集

In [13]:
run(name="chatglm_one_shot",generate_method = generate_prompt_example_title,vector_db=db,read_path=val_result_path,save_path=val_result_path)

100%|██████████| 200/200 [19:08<00:00,  5.74s/it]


,title,context,chatglm_one_shot
0,东二环精装小户13000元起 付全款96折(组图),东二环精装MINI空间——FREE自由季，主力户型56平米一居，定于5月16日盛大开盘，...,东二环精装MINI空间Free自由季5月16日盛大开盘，VIP卡办理中，96折和9.8折惊喜...
1,Adobe：IE10不支持Flash插件影响不大,新浪科技讯 北京时间9月16日早间消息，微软已表示，Windows 8的“Metro”风...,微软停止Windows 8“Metro”风格IE10浏览器Flash支持
2,詹天佑09149期福彩3D分析：胆码7 合值范围10-19,一、本人通过钟摆撞击来分析，上期开出奖号831最后的落脚点立于1数字，本期钟摆力会有所改...,
3,低调之中尽显光芒 徕卡X1送手柄15980,【5月24日太平洋电脑网浙江站】如果说当年的徕卡品牌代表着一个大众品牌，代表着摄影工业的...,徕卡X1大陆行货价格亲民 仅15980元
4,欧洲杯预选赛积分榜 德意逼近出线 法国优势缩小,北京时间6月4日凌晨，2012欧洲杯预选赛进行第6轮比赛，结果德国和意大利战胜对手提前出...,2012欧洲杯预选赛：德国和意大利提前出线，法国客场战平白俄罗斯
...,...,...,...
195,盖茨投资资产去年四季减少30亿美元,巴菲特旗下资产减值近三成 \n 缩水180亿美元；盖茨投资资产减少30亿美元 \n ...,巴菲特旗下资产减值近三成，缩水180亿美元；盖茨投资资产减少30亿美元
196,优质的壁纸是打造韩式风情的重要元素(图),80平米的两居小户型，在装饰上比较注重简约温馨的氛围，过于奢华、繁杂的风格会使其没有空间...,80平米小户型装修指南
197,整体以玫瑰色系为主(图),玫瑰花点缀洁具，整体以玫瑰色系为主，极有浪漫情调。\n\n,玫瑰点缀洁具，浪漫情调十足
198,08年全国成人高考成绩11月3日前公布,导报讯 (驻福州记者 林芹)昨日，记者从省教育厅获悉，全国成人高校招生统一考试考生成绩将...,成人高校招生统一考试成绩公布 录取线划定 集中录取和补录取时间公布


In [13]:
run(name="chatglm_zero_shot", generate_method=generate_prompt_direcet_title, read_path=val_result_path, save_path=val_result_path)

100%|██████████| 200/200 [15:29<00:00,  4.65s/it]


,title,context,chatglm_one_shot,chatglm_zero_shot
0,东二环精装小户13000元起 付全款96折(组图),东二环精装MINI空间——FREE自由季，主力户型56平米一居，定于5月16日盛大开盘，...,东二环精装MINI空间Free自由季5月16日盛大开盘，VIP卡办理中，96折和9.8折惊喜...,东二环精装MINI空间 FREE自由季盛大开盘，VIP卡办理中，11700元/m2起！5月1...
1,Adobe：IE10不支持Flash插件影响不大,新浪科技讯 北京时间9月16日早间消息，微软已表示，Windows 8的“Metro”风...,微软停止Windows 8“Metro”风格IE10浏览器Flash支持,微软停止Windows 8“Metro”风格IE10浏览器支持，Adobe仍有Flash
2,詹天佑09149期福彩3D分析：胆码7 合值范围10-19,一、本人通过钟摆撞击来分析，上期开出奖号831最后的落脚点立于1数字，本期钟摆力会有所改...,NaN,
3,低调之中尽显光芒 徕卡X1送手柄15980,【5月24日太平洋电脑网浙江站】如果说当年的徕卡品牌代表着一个大众品牌，代表着摄影工业的...,徕卡X1大陆行货价格亲民 仅15980元,徕卡X1大陆行货报价15980元，保留传统同时焕发青春
4,欧洲杯预选赛积分榜 德意逼近出线 法国优势缩小,北京时间6月4日凌晨，2012欧洲杯预选赛进行第6轮比赛，结果德国和意大利战胜对手提前出...,2012欧洲杯预选赛：德国和意大利提前出线，法国客场战平白俄罗斯,德国和意大利提前出线，法国客场战平，16支球队进入欧洲杯决赛圈
...,...,...,...,...
195,盖茨投资资产去年四季减少30亿美元,巴菲特旗下资产减值近三成 \n 缩水180亿美元；盖茨投资资产减少30亿美元 \n ...,巴菲特旗下资产减值近三成，缩水180亿美元；盖茨投资资产减少30亿美元,巴菲特和盖茨资产缩水严重：巴菲特伯克希尔投资组合缩水180亿美元，盖茨投资资产减少30亿美元
196,优质的壁纸是打造韩式风情的重要元素(图),80平米的两居小户型，在装饰上比较注重简约温馨的氛围，过于奢华、繁杂的风格会使其没有空间...,80平米小户型装修指南,80平米小户型装饰：韩式浪漫风格引领空间感
197,整体以玫瑰色系为主(图),玫瑰花点缀洁具，整体以玫瑰色系为主，极有浪漫情调。\n\n,玫瑰点缀洁具，浪漫情调十足,玫瑰点缀洁具，浪漫情调弥漫
198,08年全国成人高考成绩11月3日前公布,导报讯 (驻福州记者 林芹)昨日，记者从省教育厅获悉，全国成人高校招生统一考试考生成绩将...,成人高校招生统一考试成绩公布 录取线划定 集中录取和补录取时间公布,福建成人高校招生成绩及录取时间公布


In [14]:
run(name="chatglm_least_to_most", generate_method=generate_prompt_combined_title, read_path=val_result_path, save_path=val_result_path)

100%|██████████| 200/200 [2:07:16<00:00, 38.18s/it]  


,title,context,chatglm_one_shot,chatglm_zero_shot,chatglm_least_to_most
0,东二环精装小户13000元起 付全款96折(组图),东二环精装MINI空间——FREE自由季，主力户型56平米一居，定于5月16日盛大开盘，...,东二环精装MINI空间Free自由季5月16日盛大开盘，VIP卡办理中，96折和9.8折惊喜...,东二环精装MINI空间 FREE自由季盛大开盘，VIP卡办理中，11700元/m2起！5月1...,"""东二环精装MINI空间免费自由季盛大开盘：56平米一居付全款96折，贷款9.8折！"""
1,Adobe：IE10不支持Flash插件影响不大,新浪科技讯 北京时间9月16日早间消息，微软已表示，Windows 8的“Metro”风...,微软停止Windows 8“Metro”风格IE10浏览器Flash支持,微软停止Windows 8“Metro”风格IE10浏览器支持，Adobe仍有Flash,Adobe Flash将不支持Windows 8“Metro”风格浏览器，但与微软合作提供互...
2,詹天佑09149期福彩3D分析：胆码7 合值范围10-19,一、本人通过钟摆撞击来分析，上期开出奖号831最后的落脚点立于1数字，本期钟摆力会有所改...,NaN,NaN,彩票期号预测：华彩网分析员解读高机率号及注数，揭秘奖号预测秘密！
3,低调之中尽显光芒 徕卡X1送手柄15980,【5月24日太平洋电脑网浙江站】如果说当年的徕卡品牌代表着一个大众品牌，代表着摄影工业的...,徕卡X1大陆行货价格亲民 仅15980元,徕卡X1大陆行货报价15980元，保留传统同时焕发青春,徕卡相机焕发青春，大陆行货报价15980元，传统质量与现代感并存
4,欧洲杯预选赛积分榜 德意逼近出线 法国优势缩小,北京时间6月4日凌晨，2012欧洲杯预选赛进行第6轮比赛，结果德国和意大利战胜对手提前出...,2012欧洲杯预选赛：德国和意大利提前出线，法国客场战平白俄罗斯,德国和意大利提前出线，法国客场战平，16支球队进入欧洲杯决赛圈,德国和意大利提前出线，法国客场战平！
...,...,...,...,...,...
195,盖茨投资资产去年四季减少30亿美元,巴菲特旗下资产减值近三成 \n 缩水180亿美元；盖茨投资资产减少30亿美元 \n ...,巴菲特旗下资产减值近三成，缩水180亿美元；盖茨投资资产减少30亿美元,巴菲特和盖茨资产缩水严重：巴菲特伯克希尔投资组合缩水180亿美元，盖茨投资资产减少30亿美元,"""股神巴菲特和比尔·盖茨投资资产大幅缩水，经济危机对其影响巨大"""
196,优质的壁纸是打造韩式风情的重要元素(图),80平米的两居小户型，在装饰上比较注重简约温馨的氛围，过于奢华、繁杂的风格会使其没有空间...,80平米小户型装修指南,80平米小户型装饰：韩式浪漫风格引领空间感,韩式浪漫风格适合80平米小户型装饰：打造温馨浪漫氛围
197,整体以玫瑰色系为主(图),玫瑰花点缀洁具，整体以玫瑰色系为主，极有浪漫情调。\n\n,玫瑰点缀洁具，浪漫情调十足,玫瑰点缀洁具，浪漫情调弥漫,"""玫瑰花点缀洁具，打造浪漫家庭氛围"""
198,08年全国成人高考成绩11月3日前公布,导报讯 (驻福州记者 林芹)昨日，记者从省教育厅获悉，全国成人高校招生统一考试考生成绩将...,成人高校招生统一考试成绩公布 录取线划定 集中录取和补录取时间公布,福建成人高校招生成绩及录取时间公布,福建成人高校11月集中录取，成绩及录取线将在11月10日划定


In [15]:
# 可能因为网络等原因生成失败，这里重新把失败的append
run(name="chatglm_zero_shot", generate_method=generate_prompt_direcet_title, read_path=val_result_path, save_path=val_result_path,append=True)
run(name="chatglm_one_shot",generate_method = generate_prompt_example_title,vector_db=db,read_path=val_result_path,save_path=val_result_path,append=True)
run(name="chatglm_least_to_most", generate_method=generate_prompt_combined_title, read_path=val_result_path, save_path=val_result_path,append=True)

100%|██████████| 200/200 [04:50<00:00,  1.45s/it]


,title,context,chatglm_one_shot,chatglm_zero_shot,chatglm_least_to_most
0,东二环精装小户13000元起 付全款96折(组图),东二环精装MINI空间——FREE自由季，主力户型56平米一居，定于5月16日盛大开盘，...,东二环精装MINI空间Free自由季5月16日盛大开盘，VIP卡办理中，96折和9.8折惊喜...,东二环精装MINI空间 FREE自由季盛大开盘，VIP卡办理中，11700元/m2起！5月1...,"""东二环精装MINI空间免费自由季盛大开盘：56平米一居付全款96折，贷款9.8折！"""
1,Adobe：IE10不支持Flash插件影响不大,新浪科技讯 北京时间9月16日早间消息，微软已表示，Windows 8的“Metro”风...,微软停止Windows 8“Metro”风格IE10浏览器Flash支持,微软停止Windows 8“Metro”风格IE10浏览器支持，Adobe仍有Flash,Adobe Flash将不支持Windows 8“Metro”风格浏览器，但与微软合作提供互...
2,詹天佑09149期福彩3D分析：胆码7 合值范围10-19,一、本人通过钟摆撞击来分析，上期开出奖号831最后的落脚点立于1数字，本期钟摆力会有所改...,NaN,NaN,彩票期号预测：华彩网分析员解读高机率号及注数，揭秘奖号预测秘密！
3,低调之中尽显光芒 徕卡X1送手柄15980,【5月24日太平洋电脑网浙江站】如果说当年的徕卡品牌代表着一个大众品牌，代表着摄影工业的...,徕卡X1大陆行货价格亲民 仅15980元,徕卡X1大陆行货报价15980元，保留传统同时焕发青春,徕卡相机焕发青春，大陆行货报价15980元，传统质量与现代感并存
4,欧洲杯预选赛积分榜 德意逼近出线 法国优势缩小,北京时间6月4日凌晨，2012欧洲杯预选赛进行第6轮比赛，结果德国和意大利战胜对手提前出...,2012欧洲杯预选赛：德国和意大利提前出线，法国客场战平白俄罗斯,德国和意大利提前出线，法国客场战平，16支球队进入欧洲杯决赛圈,德国和意大利提前出线，法国客场战平！
...,...,...,...,...,...
195,盖茨投资资产去年四季减少30亿美元,巴菲特旗下资产减值近三成 \n 缩水180亿美元；盖茨投资资产减少30亿美元 \n ...,巴菲特旗下资产减值近三成，缩水180亿美元；盖茨投资资产减少30亿美元,巴菲特和盖茨资产缩水严重：巴菲特伯克希尔投资组合缩水180亿美元，盖茨投资资产减少30亿美元,"""股神巴菲特和比尔·盖茨投资资产大幅缩水，经济危机对其影响巨大"""
196,优质的壁纸是打造韩式风情的重要元素(图),80平米的两居小户型，在装饰上比较注重简约温馨的氛围，过于奢华、繁杂的风格会使其没有空间...,80平米小户型装修指南,80平米小户型装饰：韩式浪漫风格引领空间感,韩式浪漫风格适合80平米小户型装饰：打造温馨浪漫氛围
197,整体以玫瑰色系为主(图),玫瑰花点缀洁具，整体以玫瑰色系为主，极有浪漫情调。\n\n,玫瑰点缀洁具，浪漫情调十足,玫瑰点缀洁具，浪漫情调弥漫,"""玫瑰花点缀洁具，打造浪漫家庭氛围"""
198,08年全国成人高考成绩11月3日前公布,导报讯 (驻福州记者 林芹)昨日，记者从省教育厅获悉，全国成人高校招生统一考试考生成绩将...,成人高校招生统一考试成绩公布 录取线划定 集中录取和补录取时间公布,福建成人高校招生成绩及录取时间公布,福建成人高校11月集中录取，成绩及录取线将在11月10日划定


###